In [25]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

In [39]:
def string_to_timestamp(strings,t=48):
    """
    :param strings: [b'2013070101,b'2013070102']
    :param t: 48 timestamp of a day 
    :return: '2013-0701 00:00:00','2013-07-01 00:30:00'
    """
    timestamps = []
    time_per_slot = 24.0 / t
    num_per_t = t
    
    for t in strings:
        print(t)
        year, month, day, slot = int(t[:4]), int(t[4:6]), int(t[6:8]), int(t[8:]) - 1 
        timestamps.append(pd.Timestamp(datetime(year,month,day,hour = int(slot * time_per_slot),minute = (slot % num_per_t)* int (60.0 * time_per_slot))))    
    
    return timestamps
# example:
print(string_to_timestamp(["2013070101"]))

2013070101
[Timestamp('2013-07-01 00:00:00')]


In [40]:
def timestamp_to_vec(timestamps):
    """
    时间戳_to_vec
    :param timestamps: 
    :return: vec
    example:
    ['2018120505','2018120106']
    return：[[0 0 1 0 0 0 0 1]当前是星期三，工作日
            [0 0 0 0 0 1 0 0]当前是星期六，休息日                             
    """
    vec = [time.strptime(str(t[:8]),'%Y%m%d').tm_wday for t in timestamps]
    ret = []
    for i in vec:
        v = [0 for j in range(7)]
        v[i] = 1
        if i >= 5:
            v.append(0) # weekend
        else:
            v.append(1) # weekday
        ret.append(v)
    return np.asarray(ret)
# example:
print(timestamp_to_vec(['2018120505']))
            


[[0 0 1 0 0 0 0 1]]


In [ ]:
class STMatrix(object):
    """
    string for STMatrix
    
    """
    def __init__(self, data, timestamps, t=48, check_complete=True):
        super(STMatrix,self).__init__()
        self.get_index = dict()
        self.make_index()
        assert len(data) == len(timestamps)
        self.data = data
        self.timestamps = timestamps
        self.t = t
        self.get_index
        self.pd_timestamps = string_to_timestamp(timestamps,T = self.T)
        if check_complete:
            self.check_complete()
        self.make_index()
        
    def make_index(self):
        for i,ts in enumerate(self.pd_timestamps):
            self.get_index[ts] = i
    def check_complete(self):
        missing_timestamps = []
        offset = pd.DateOffset(minutes = 24*60)
        pd_timestamps = self.pd_timestamps
        i = 1
        while i < len(pd_timestamps):
            if pd_timestamps[i-1] + offset != pd_timestamps[i]:
                missing_timestamps.append('(%s -- %s)' % (pd_timestamps[i-1],pd_timestamps[i]))
            i += 1
    def get_matrix(self, timestamp):
            return self.data[self.get_index[timestamp]]
    def check_it(self, depends):
        for d in depends:
            if d not in self.get_index.keys():
                return False
        return True
    
    def create_dataset(self,len_cloneness=3, len_trend=3, trend_interval=7,len_period=3,period_interval=1):
        offset_frame = pd.DateOffset(minutes = 24*60 // self.T) # 时间偏移
        XC = []
        XP = []
        XT = []
        Y = []
        timestamps_Y = []
        depends = [range(1,len_cloneness),
                   [period_interval*self.t*j for j in range(1,len_period+1)],
                   [trend_interval*self.t*j for j in range(1,len_trend+1)]]
        i = max(self.t*trend_interval * len_trend,self.t*period_interval*len_period,len_cloneness)
        

<DateOffset: minutes=1440>
